In [23]:
import os
from dotenv import load_dotenv
from datetime import datetime
import re
import pandas as pd
import psycopg2
import requests
import tiktoken

load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DEB_NAME")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Carregando tabelas do DB
# Definir os parâmetros de conexão com o PostgreSQL
usuario = DB_USER
senha = DB_PASS
host = DB_HOST  
porta = DB_PORT           
banco_de_dados = DB_NAME

modelo = "gpt-4"

In [24]:

# Exemplo de uso da função
# salvar_dados_com_condicao('sua_tabela', 'coluna1', 'valor1', 'coluna2', 'valor2')


In [25]:
def contar_tokens(texto: str, modelo: str) -> int:
    """
    Conta o número de tokens em um texto para o modelo especificado.

    Args:
        texto (str): O texto para contar os tokens.
        modelo (str): O nome do modelo (ex: "gpt-3.5-turbo", "gpt-4").

    Returns:
        int: O número de tokens contados.
    """
    # Cria um codificador para o modelo escolhido
    encoding = tiktoken.encoding_for_model(modelo)

    # Conta os tokens
    tokens = encoding.encode(texto)
    num_tokens = len(tokens)

    return num_tokens
    
# Função para criar um DataFrame a partir de uma tabela do banco de dados
def carregar_tabela_postgresql(nome_tabela):
    try:
        # Estabelecendo conexão com o banco de dados
        conn = psycopg2.connect(
            dbname=banco_de_dados,
            user=usuario,
            password=senha,
            host=host,
            port=porta
        )
        
        # Query para selecionar todos os dados da tabela desejada
        query = f"SELECT * FROM {nome_tabela};"
        
        # Lendo os dados diretamente no DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Fechando a conexão
        conn.close()
        
        return df
    
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

## Função para criar um DataFrame a partir de uma tabela do banco de dados, permitindo parametrizar a condição WHERE
def carregar_tabela_com_where(nome_tabela, where_condition):
    try:
        # Estabelecendo conexão com o banco de dados
        conn = psycopg2.connect(
            dbname=banco_de_dados,
            user=usuario,
            password=senha,
            host=host,
            port=porta
        )
        
        # Query para selecionar os dados da tabela desejada com a condição WHERE
        query = f"SELECT * FROM {nome_tabela} WHERE {where_condition};"
        
        # Lendo os dados diretamente no DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Fechando a conexão
        conn.close()
        
        return df
    
    except Exception as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

def atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, valor_atualizar, coluna_condicao, valor_condicao):
    try:
        # Estabelecendo conexão com o banco de dados
        conn = psycopg2.connect(
            dbname=banco_de_dados,
            user=usuario,
            password=senha,
            host=host,
            port=porta
        )
        
        # Criando um cursor para executar a query
        cursor = conn.cursor()

        # Query para atualizar os dados
        query = f"""
        UPDATE {nome_tabela} 
        SET {coluna_atualizar} = '{valor_atualizar}' 
        WHERE {coluna_condicao} = '{valor_condicao}';
        """
        print(query)
        # Executando a query com os valores fornecidos
        cursor.execute(query, (valor_atualizar, valor_condicao))
        
        # Commitando a transação
        conn.commit()
        
        # Fechando o cursor e a conexão
        cursor.close()
        conn.close()
        
        print(f"Dados atualizados com sucesso na tabela {nome_tabela}.")
    
    except Exception as e:
        print(f"Erro ao atualizar dados na tabela {nome_tabela}: {e}")


# chat_history = carregar_tabela_com_where("chat_history_t",f"created_at::date = (CURRENT_DATE - INTERVAL '{relatorio_dia} day')") #Coreto = 1 day
chat_history = carregar_tabela_com_where("chat_history",f"created_at::date = '2024-10-15'") #Coreto = 1 day
people = carregar_tabela_postgresql("people")
groups = carregar_tabela_postgresql("groups")
packages = carregar_tabela_postgresql("packages")
departments = carregar_tabela_postgresql("departments")
projects = carregar_tabela_postgresql("projects")
stages = carregar_tabela_postgresql("stages")

/tmp/ipykernel_273621/3857067134.py:64: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/tmp/ipykernel_273621/3857067134.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [26]:
# Função para criar um novo thread_id
def create_thread():
    url = "https://api.openai.com/v1/threads"
    headers = {
        'Authorization': f'Bearer {OPENAI_API_KEY}',  # Substitua pela sua chave de API
        'Content-Type': 'application/json',
        'OpenAI-Beta': 'assistants=v2',
    }
    response = requests.post(url, headers=headers)
    
    if response.status_code == 200:
        return response.json().get('id')  # Retorna o thread_id criado
    else:
        print(f"Erro ao criar thread: {response.status_code} - {response.text}")
        return None

# Iterar pelas linhas do DataFrame e criar thread_id se estiver vazio
for index, row in groups.iterrows():
    if pd.isna(row['thread_id']):
        new_thread_id = create_thread()
        if new_thread_id:
            groups.at[index, 'thread_id'] = new_thread_id  # Atualiza o DataFrame com o novo thread_id
            group_jid = row['group_jid']
            thread_id = row['thread_id']
            atualizar_dados_com_condicao('groups','thread_id',new_thread_id,'group_jid',group_jid)
# atualizar_dados_com_condicao('sua_tabela', 'coluna_atualizar', 'novo_valor', 'coluna_condicao', 'valor_condicao')

In [27]:
# print(groups.columns)
print(projects.columns)
# print(stages.columns)
# projects = projects.rename(columns={'status': 'project_status', 'description':'project_description', 'created_at': 'project_created',  'updated_at': 'project_updated', 'estimated_deadline':'project_estimated_deadline', 'closed_at':'project_closed_at'})
# projects_cols = ['project_id', 'package_id', 'project_name', 'welcome_sent', 'timeline_sent', 'niche', 'products_categories', 'layout_references',]
# projects = projects[projects_cols]

Index(['project_id', 'package_id', 'stage_id', 'project_name', 'welcome_sent',
       'timeline_sent', 'niche', 'products_categories', 'layout_references',
       'shop_colors', 'logo_style', 'font_style', 'payment_platforms',
       'products_platforms', 'social_media_info', 'ecommerce_platform',
       'ecommerce_user', 'ecommerce_pass', 'domain', 'domain_platform',
       'domain_user', 'domain_pass', 'facebook_user', 'facebook_pass',
       'instagram_user', 'instagram_pass', 'logo', 'mined_products', 'pricing',
       'product_ad1', 'product_ad2', 'ecommerce_call', 'marketing_call',
       'had_marketing_call', 'ecommerce_transfer', 'meta_ads_config',
       'creative_approved', 'publication_campaign',
       'report_marketing_conclusion', 'conclusion_marketing_call',
       'campaign_optimization', 'offer_complementary_services', 'status',
       'description', 'created_at', 'updated_at', 'estimated_deadline',
       'closed_at'],
      dtype='object')


In [28]:
# Identifica projetos, grupos e etapas

groups_verify = groups.rename(columns={'description': 'group_description', 'created_at': 'group_created', 'estimated_deadline':'group_estimated_deadline', 'status':'group_status'})
groups_cols = ['group_id', 'group_jid', 'group_name', 'project_id', 'group_status', 'thread_id', 'group_description', 'group_created']
groups_verify = groups_verify[groups_cols]

projects = projects.rename(columns={'status': 'project_status', 'description':'project_description', 'created_at': 'project_created',  'updated_at': 'project_updated', 'estimated_deadline':'project_estimated_deadline', 'closed_at':'project_closed_at'})
# projects_cols = ['project_id', 'package_id', 'project_name', 'welcome_sent', 'timeline_sent', 'niche', 'products_categories', 'layout_references',]
# projects = projects[projects_cols]

stages = stages.rename(columns={'description': 'stage_description', 'created_at': 'stage_created', 'estimated_deadline':'stage_estimated_deadline'})
stages_cols = ['stage_id', 'package_id', 'stage', 'stage_name', 'stage_description']
stages = stages[stages_cols]

# Unindo projetos e grupos
projects_groups = pd.merge(
    projects,
    groups_verify,
    left_on='project_id',
    right_on='project_id',
    how='left'  # Tipo de merge
)

projects_groups

# Unindo projetos e etapas
projects_stages = pd.merge(
    projects_groups,  # DataFrame com os projetos
    stages,  # DataFrame com as etapas
    left_on=['stage_id', 'package_id'],  # Colunas de chave no DataFrame 'projects'
    right_on=['stage_id', 'package_id'],  # Colunas de chave no DataFrame 'stages'
    how='left'  # Mantém todos os dados de 'projects' mesmo que não haja correspondência em 'stages'
)

projects_stages

,project_id,package_id,stage_id,project_name,welcome_sent,timeline_sent,niche,products_categories,layout_references,shop_colors,...,group_id,group_jid,group_name,group_status,thread_id,group_description,group_created,stage,stage_name,stage_description
0,1,1,7,GT_01 15/10,True,True,Novo Nicho Definido,Novas Categorias Definidas,Novas Referências de Layout,Cores Atualizadas,...,1,120363330397856732@g.us,GT_01 15/10,ativo,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Grupo de testes,2024-10-15 09:17:00.369336,7,Etapa 7,Descrição da Etapa 7


In [29]:
projects_stages.columns

Index(['project_id', 'package_id', 'stage_id', 'project_name', 'welcome_sent',
       'timeline_sent', 'niche', 'products_categories', 'layout_references',
       'shop_colors', 'logo_style', 'font_style', 'payment_platforms',
       'products_platforms', 'social_media_info', 'ecommerce_platform',
       'ecommerce_user', 'ecommerce_pass', 'domain', 'domain_platform',
       'domain_user', 'domain_pass', 'facebook_user', 'facebook_pass',
       'instagram_user', 'instagram_pass', 'logo', 'mined_products', 'pricing',
       'product_ad1', 'product_ad2', 'ecommerce_call', 'marketing_call',
       'had_marketing_call', 'ecommerce_transfer', 'meta_ads_config',
       'creative_approved', 'publication_campaign',
       'report_marketing_conclusion', 'conclusion_marketing_call',
       'campaign_optimization', 'offer_complementary_services',
       'project_status', 'project_description', 'project_created',
       'project_updated', 'project_estimated_deadline', 'project_closed_at',
       '

In [30]:
# Função para realizar diferentes ações com base no valor de 'stage'
def verificar_stage_e_atualizar(row):
    nome_tabela = "projects"
    coluna_atualizar = "stage_id"
    coluna_condicao = "project_id"
    valor_condicao = row['project_id']  
    
    if row['stage'] == 1:
        checks = [
            row['welcome_sent'],
            row['timeline_sent'],
            row['niche'],
            row['products_categories'],
            row['layout_references'],
            row['shop_colors'],
            row['logo_style'],
            row['font_style'],
            row['payment_platforms'],
            row['products_platforms'],
            row['social_media_info'],
            row['ecommerce_platform'],
            row['ecommerce_user'],
            row['ecommerce_pass'],
            row['domain'],
            row['domain_platform'],
            row['domain_user'],
            row['domain_pass']
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 2
            atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, 2, coluna_condicao, valor_condicao)
    elif row['stage'] == 2:
        checks = [
            row['facebook_user'], 
            row['facebook_pass'],
            row['instagram_user'], 
            row['instagram_pass']
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 3
            atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, 3, coluna_condicao, valor_condicao)
    
    elif row['stage'] == 3:
        checks = [
            row['logo'], 
            row['mined_products'], 
            row['pricing'], 
            row['product_ad1'], 
            row['product_ad2'], 
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 4
            atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, 4, coluna_condicao, valor_condicao)

    elif row['stage'] == 4:
        checks = [
            row['ecommerce_call'],
            row['marketing_call'] 
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 5
            atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, 5, coluna_condicao, valor_condicao)

    elif row['stage'] == 5:
        checks = [
            row['had_marketing_call'],
            row['ecommerce_transfer'] 
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 6
            atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, 6, coluna_condicao, valor_condicao)

    elif row['stage'] == 6:
        checks = [
            row['meta_ads_config'],
            row['creative_approved'],
            row['publication_campaign'],
            row['report_marketing_conclusion']
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 7
            atualizar_dados_com_condicao(nome_tabela, coluna_atualizar, 7, coluna_condicao, valor_condicao)

    elif row['stage'] == 7:
        checks = [
            row['conclusion_marketing_call'],
            row['campaign_optimization'],
            row['offer_complementary_services']
        ]
        
        if all(checks):
            # Atualiza o campo 'stage' para 8 (ou alguma outra ação final)
            atualizar_dados_com_condicao(nome_tabela, 'status', 'Finalizando', coluna_condicao, valor_condicao)

# Aplicando a função a cada linha do DataFrame
projects_stages.apply(verificar_stage_e_atualizar, axis=1)


        UPDATE projects 
        SET status = 'Finalizando' 
        WHERE project_id = '1';
        
Dados atualizados com sucesso na tabela projects.


0    None
dtype: object

In [31]:
groups

,group_id,group_jid,group_name,package_id,project_id,status,thread_id,description,created_at,updated_at
0,1,120363330397856732@g.us,GT_01 15/10,1,1,ativo,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Grupo de testes,2024-10-15 09:17:00.369336,2024-10-15 09:17:00.369343


In [32]:
# grupos
groups = groups.rename(columns={'description': 'group_description', 'created_at': 'group_created'})
groups_cols = ['group_id','group_jid', 'group_name', 'package_id', 'project_id', 'group_description', 'group_created','thread_id']
groups = groups[groups_cols]

# pacotes
packages = packages.rename(columns={'description': 'package_description'})
packages_cols = ['package_id', 'package_name']
packages = packages[packages_cols]

# pessoas
people_cols = ['remote_jid', 'person_name','nickname','department_id','is_client']
people = people[people_cols]

# departamentos
departments = departments.rename(columns={'description': 'department_description'})
departments_cols = ['department_id', 'department_name','department_description']
departments = departments[departments_cols]

# Unindo pessoas e departamentos
person_dep = pd.merge(people, departments, on='department_id', how='left')
person_dep_cols = ['remote_jid', 'person_name','nickname','is_client','department_name']
person_dep = person_dep[person_dep_cols]

# Unindo historico e pessoas
chat_history = pd.merge(
    chat_history,
    person_dep,
    left_on='sender_id',  # Coluna do DataFrame chat_history
    right_on='remote_jid',  # Coluna do DataFrame person_dep
    how='left'  # Tipo de merge
)

# Unindo gupos e pacotes
grupos_pacotes = pd.merge(groups, packages, on='package_id', how='left')

# Unindo historico e grupos
chat_groups_history = pd.merge(
    chat_history,
    grupos_pacotes,
    left_on='groupJid',  # Coluna do DataFrame chat_history
    right_on='group_jid',  # Coluna do DataFrame person_dep
    how='left'  # Tipo de merge
)

chat_groups_history = chat_groups_history.rename(columns={'created_at': 'ch_created_at'})
chat_gh_cols = ['instance', 'receiver', 'sender_id','push_name','base64_data','sequence_number','group_jid','package_id','group_description','from_me','event']
chat_groups_history = chat_groups_history.drop(columns=chat_gh_cols)



In [33]:
chat_groups_history.columns

Index(['message_id', 'groupJid', 'message_type', 'message_content',
       'media_url', 'reaction', 'latitude', 'longitude', 'display_name',
       'vcard', 'poll_data', 'source', 'reaction_message_id', 'quoted_message',
       'ch_created_at', 'remote_jid', 'person_name', 'nickname', 'is_client',
       'department_name', 'group_id', 'group_name', 'project_id',
       'group_created', 'thread_id', 'package_name'],
      dtype='object')

In [34]:
dfs_groupJid = {}

#  Obter todos os valores únicos na coluna groupJid
grupos = chat_groups_history['groupJid'].unique()

# Para cada valor, filtrar e armazenar em um novo DataFrame
for g in grupos:
    df_grupos = chat_groups_history[chat_groups_history['groupJid'] == g]
    df_grupos = df_grupos.sort_values(by='ch_created_at', ascending=True)
    dfs_groupJid[g] = df_grupos
    print(g)

None
120363330397856732@g.us


In [35]:
df_sample = dfs_groupJid['120363330397856732@g.us'].copy()

# Obter valores 
group_id = df_sample['group_id'].unique().tolist()
group_id = group_id[0]

group_jid = df_sample['groupJid'].unique().tolist()
group_jid = group_jid[0]

group_name = df_sample['group_name'].unique().tolist()
group = group_name[0]

package_name = df_sample['package_name'].unique().tolist()
package = package_name[0]

print(group_id)
print(group_jid)
print(group)
print(package)

1.0
120363330397856732@g.us
GT_01 15/10
Pacote Especial


In [36]:
df_sample.head()

,message_id,groupJid,message_type,message_content,media_url,reaction,latitude,longitude,display_name,vcard,...,person_name,nickname,is_client,department_name,group_id,group_name,project_id,group_created,thread_id,package_name
1,3EB0E4A15A010853D903BA,120363330397856732@g.us,conversation,Sejam bem vind@s ao nosso grupo de testes \n,None,None,None,\n,web,None,...,Luiz Paulo Coutinho,LpCoutinho,False,Desenvolvimento,1.0,GT_01 15/10,1.0,2024-10-15 09:17:00.369336,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Pacote Especial
2,3EB0E4A15A010853D903BA_deleted,120363330397856732@g.us,None,None,None,None,None,\n,None,None,...,NaN,NaN,NaN,NaN,1.0,GT_01 15/10,1.0,2024-10-15 09:17:00.369336,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Pacote Especial
3,3EB0097778FD9C83635EB4,120363330397856732@g.us,conversation,Sejam bem vind@s ao nosso grupo de testes,None,None,None,\n,web,None,...,Luiz Paulo Coutinho,LpCoutinho,False,Desenvolvimento,1.0,GT_01 15/10,1.0,2024-10-15 09:17:00.369336,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Pacote Especial
4,C7D99D510F210D0E73AA113AACCD5DE5,120363330397856732@g.us,conversation,Sejam bem vind@s ao nosso grupo de teste,None,None,None,\n,android,None,...,Luiz Paulo Coutinho,LpCoutinho,False,Desenvolvimento,1.0,GT_01 15/10,1.0,2024-10-15 09:17:00.369336,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Pacote Especial
5,6C7C85662D433BE225AE688F80E7786E,120363330397856732@g.us,editedMessage,Sejam bem vind@s ao nosso grupo de testes,None,None,None,\n,android,None,...,Luiz Paulo Coutinho,LpCoutinho,False,Desenvolvimento,1.0,GT_01 15/10,1.0,2024-10-15 09:17:00.369336,thread_z9jiEg4XLRxLzUTh8Cp2kRXa,Pacote Especial


In [37]:
projects['project_id']

0    1
Name: project_id, dtype: int64

In [38]:
# Verifique se 'project_id' existe em ambos os DataFrames
if 'project_id' in projects.columns and 'project_id' in df_sample.columns:
    # Filtra o DataFrame projects para incluir apenas os project_ids que estão em df_sample
    sample_project = projects[projects['project_id'].isin(df_sample['project_id'])]
else:
    print("A coluna 'project_id' não existe em um dos DataFrames.")


In [39]:
sample_project.columns

Index(['project_id', 'package_id', 'stage_id', 'project_name', 'welcome_sent',
       'timeline_sent', 'niche', 'products_categories', 'layout_references',
       'shop_colors', 'logo_style', 'font_style', 'payment_platforms',
       'products_platforms', 'social_media_info', 'ecommerce_platform',
       'ecommerce_user', 'ecommerce_pass', 'domain', 'domain_platform',
       'domain_user', 'domain_pass', 'facebook_user', 'facebook_pass',
       'instagram_user', 'instagram_pass', 'logo', 'mined_products', 'pricing',
       'product_ad1', 'product_ad2', 'ecommerce_call', 'marketing_call',
       'had_marketing_call', 'ecommerce_transfer', 'meta_ads_config',
       'creative_approved', 'publication_campaign',
       'report_marketing_conclusion', 'conclusion_marketing_call',
       'campaign_optimization', 'offer_complementary_services',
       'project_status', 'project_description', 'project_created',
       'project_updated', 'project_estimated_deadline', 'project_closed_at'],
      d

In [40]:
# Supondo que sample_project tenha apenas uma linha
project = sample_project.iloc[0].to_dict()

# Armazenar todas as variáveis do projeto
project_id = project['project_id']
package_id = project['package_id']
stage_id = project['stage_id']
project_name = project['project_name']
welcome_sent = project['welcome_sent']
timeline_sent = project['timeline_sent']
niche = project['niche']
products_categories = project['products_categories']
layout_references = project['layout_references']
shop_colors = project['shop_colors']
logo_style = project['logo_style']
font_style = project['font_style']
payment_platforms = project['payment_platforms']
products_platforms = project['products_platforms']
social_media_info = project['social_media_info']
ecommerce_platform = project['ecommerce_platform']
ecommerce_user = project['ecommerce_user']
ecommerce_pass = project['ecommerce_pass']
domain = project['domain']
domain_platform = project['domain_platform']
domain_user = project['domain_user']
domain_pass = project['domain_pass']
facebook_user = project['facebook_user']
facebook_pass = project['facebook_pass']
instagram_user = project['instagram_user']
instagram_pass = project['instagram_pass']
logo = project['logo']
mined_products = project['mined_products']
pricing = project['pricing']
product_ad1 = project['product_ad1']
product_ad2 = project['product_ad2']
ecommerce_call = project['ecommerce_call']
marketing_call = project['marketing_call']
had_marketing_call = project['had_marketing_call']
ecommerce_transfer = project['ecommerce_transfer']
meta_ads_config = project['meta_ads_config']
creative_approved = project['creative_approved']
publication_campaign = project['publication_campaign']
report_marketing_conclusion = project['report_marketing_conclusion']
conclusion_marketing_call = project['conclusion_marketing_call']
campaign_optimization = project['campaign_optimization']
offer_complementary_services = project['offer_complementary_services']
project_status = project['project_status']
project_description = project['project_description']
project_created = project['project_created']
project_updated = project['project_updated']
project_estimated_deadline = project['project_estimated_deadline']
project_closed_at = project['project_closed_at']

# Exemplo de uso das variáveis
print(f"Project ID: {project_id}")
print(f"Project Name: {project_name}")


Project ID: 1
Project Name: GT_01 15/10


In [41]:
# Supondo que project tenha sido criado e contém os dados
stage_id = project['stage_id']

# Variáveis para cada etapa
if stage_id == 1:
    welcome_sent = project['welcome_sent']
    timeline_sent = project['timeline_sent']
    niche = project['niche']
    products_categories = project['products_categories']
    layout_references = project['layout_references']
    shop_colors = project['shop_colors']
    logo_style = project['logo_style']
    font_style = project['font_style']
    payment_platforms = project['payment_platforms']
    products_platforms = project['products_platforms']
    social_media_info = project['social_media_info']
    ecommerce_platform = project['ecommerce_platform']
    ecommerce_user = project['ecommerce_user']
    ecommerce_pass = project['ecommerce_pass']
    domain = project['domain']
    domain_platform = project['domain_platform']
    domain_user = project['domain_user']
    domain_pass = project['domain_pass']

elif stage_id == 2:
    facebook_user = project['facebook_user']
    facebook_pass = project['facebook_pass']
    instagram_user = project['instagram_user']
    instagram_pass = project['instagram_pass']

elif stage_id == 3:
    logo = project['logo']
    mined_products = project['mined_products']
    pricing = project['pricing']
    product_ad1 = project['product_ad1']
    product_ad2 = project['product_ad2']

elif stage_id == 4:
    ecommerce_call = project['ecommerce_call']
    marketing_call = project['marketing_call']

elif stage_id == 5:
    had_marketing_call = project['had_marketing_call']
    ecommerce_transfer = project['ecommerce_transfer']

elif stage_id == 6:
    meta_ads_config = project['meta_ads_config']
    creative_approved = project['creative_approved']
    publication_campaign = project['publication_campaign']
    report_marketing_conclusion = project['report_marketing_conclusion']

elif stage_id == 7:
    conclusion_marketing_call = project['conclusion_marketing_call']
    campaign_optimization = project['campaign_optimization']
    offer_complementary_services = project['offer_complementary_services']

else:
    print("Stage ID inválido.")    

In [49]:
from datetime import datetime

# Data de hoje
today = datetime.now().strftime('%d/%m/%Y %H:%M')

# Supondo que você tenha as seguintes variáveis já definidas:
# project_created, project_updated, project_estimated_deadline, package_id, stage_id

# Mensagem base
mensagem = f"""
Projeto iniciado em {project_created.strftime('%d/%m/%Y %H:%M')}
Estimativa de conclusão em {project_estimated_deadline.strftime('%d/%m/%Y %H:%M')}
Data de hoje: {today}
Pacote: {package_id}
Etapa: {stage_id}/7

CheckList:
"""

# Adicionando variáveis conforme o estágio
if stage_id == 1:
    checklist_items = [
        f"Bem-vindo enviado: {welcome_sent}",
        f"Timeline enviada: {timeline_sent}",
        f"Nicho: {niche}",
        f"Categorias de produtos: {products_categories}",
        f"Referências de layout: {layout_references}",
        f"Cores da loja: {shop_colors}",
        f"Estilo do logo: {logo_style}",
        f"Estilo da fonte: {font_style}",
        f"Plataformas de pagamento: {payment_platforms}",
        f"Plataformas de produtos: {products_platforms}",
        f"Informações de mídias sociais: {social_media_info}",
        f"Plataforma de e-commerce: {ecommerce_platform}",
        f"Usuário de e-commerce: {ecommerce_user}",
        f"Senha de e-commerce: {ecommerce_pass}",
        f"Domínio: {domain}",
        f"Plataforma de domínio: {domain_platform}",
        f"Usuário de domínio: {domain_user}",
        f"Senha de domínio: {domain_pass}",
    ]
    
elif stage_id == 2:
    checklist_items = [
        f"Usuário do Facebook: {facebook_user}",
        f"Senha do Facebook: {facebook_pass}",
        f"Usuário do Instagram: {instagram_user}",
        f"Senha do Instagram: {instagram_pass}",
    ]
    
elif stage_id == 3:
    checklist_items = [
        f"Logo: {logo}",
        f"Produtos minerados: {mined_products}",
        f"Precificação: {pricing}",
        f"Anúncio do produto 1: {product_ad1}",
        f"Anúncio do produto 2: {product_ad2}",
    ]
    
elif stage_id == 4:
    checklist_items = [
        f"Chamada de e-commerce: {ecommerce_call}",
        f"Chamada de marketing: {marketing_call}",
    ]
    
elif stage_id == 5:
    checklist_items = [
        f"Teve chamada de marketing: {had_marketing_call}",
        f"Transferência de e-commerce: {ecommerce_transfer}",
    ]
    
elif stage_id == 6:
    checklist_items = [
        f"Configuração de meta ads: {meta_ads_config}",
        f"Aprovado criativo: {creative_approved}",
        f"Campanha de publicação: {publication_campaign}",
        f"Conclusão do relatório de marketing: {report_marketing_conclusion}",
    ]
    
elif stage_id == 7:
    checklist_items = [
        f"Conclusão da chamada de marketing: {conclusion_marketing_call}",
        f"Otimização de campanha: {campaign_optimization}",
        f"Oferta de serviços complementares: {offer_complementary_services}",
    ]
    
else:
    checklist_items = ["Etapa inválida."]

# Adicionando checklist na mensagem
mensagem += "\n" + "\n".join(checklist_items)

# # Análise da conversa no WhatsApp
# mensagem += "\n\nAnalisando a conversa no WhatsApp e verificando se os itens no checklist estão sendo abordados e preenchidos. Caso algum item ainda não esteja, favor atualizar."

# Mensagens específicas por etapa
mensagem_final = ""
try:
    with open(f'etapa{stage_id}.txt', 'r') as file:
        mensagem_final = file.read()
except FileNotFoundError:
    mensagem_final = "Mensagem específica para essa etapa não encontrada."

mensagem += "\n\n" + mensagem_final

# # 2. Transformar o DataFrame em JSON
# rem_cols = ['thread_id']
# df_sample = df_sample.drop(columns=rem_cols)
json_data = df_sample.to_json(orient='records')


mensagem +="\n\n" + "A conversa do dia:" + "\n\n" + json_data
# Exibir mensagem final
print(mensagem)



Projeto iniciado em 15/10/2024 09:17
Estimativa de conclusão em 14/11/2024 09:17
Data de hoje: 15/10/2024 18:24
Pacote: 1
Etapa: 7/7

CheckList:

Conclusão da chamada de marketing: true
Otimização de campanha: True
Oferta de serviços complementares: true

**Tarefa**: Gere um relatório detalhado para a Etapa Final do projeto, com base na descrição a seguir. O relatório deve incluir todos os pontos-chave da etapa, organizados de forma clara e concisa, ressaltando os objetivos da call de conclusão e finalização do projeto.

**Descrição da Etapa Final**:
Na Etapa Final, considerada a última etapa do projeto, será feito o agendamento da call de conclusão e finalização do projeto. Esta call será conduzida pelo atendente responsável e pelo gestor de marketing. O foco da call terá 3 objetivos principais:

1. **Efetivação da Entrega da Loja**:
   - Concluir a entrega da loja e esclarecer todas as dúvidas do cliente para a pós-finalização total do projeto, utilizando um script padrão com os ass

In [48]:
num_tokens = contar_tokens(mensagem, modelo)
print(f"Total de tokens: {num_tokens}")

Total de tokens: 5421


# Cria mensagem com historico em json

In [ ]:
# 1. Definir o thread_id
thread_id = df_sample['thread_id'].unique().item()
url = f'https://api.openai.com/v1/threads/{thread_id}/messages' # cria mensagem

# 2. Transformar o DataFrame em JSON
rem_cols = ['thread_id']
df_sample = df_sample.drop(columns=rem_cols)
json_data = df_sample.to_json(orient='records')
json_data

# 3. Configurar o cabeçalho da requisição
headers = {
    'Authorization': f'Bearer {OPENAI_API_KEY}',  # Substitua pela sua chave de API
    'Content-Type': 'application/json',
    'OpenAI-Beta': 'assistants=v2',
}

# 4. Configurar o corpo da requisição
body = {
    'role': 'user',
    'content': json_data,
}

# 5. Enviar a requisição POST com o corpo em JSON
response = requests.post(url, headers=headers, json=body)

# 6. Verificar a resposta
if response.status_code == 200:
    print("Dados enviados com sucesso:", response.json())
else:
    print("Erro ao enviar dados:", response.status_code, response.text)

# Roda Assitant

In [ ]:
# 1. Definir url
url = f'https://api.openai.com/v1/threads/{thread_id}/runs'

# 4. Configurar o corpo da requisição
body = {
    'assistant_id': 'asst_B0A4BK286pd9Kj9zKXzNDfdR',
}

# 5. Enviar a requisição POST com o corpo em JSON
response = requests.post(url, headers=headers, json=body)

# 6. Verificar a resposta
# Supondo que 'response' é a resposta da requisição POST já verificada
if response.status_code == 200:
    data = response.json()
    
    # Extraindo os parâmetros em variáveis
    run_id = data.get('id')
    object_type = data.get('object')
    created_at = data.get('created_at')
    assistant_id = data.get('assistant_id')
    thread_id = data.get('thread_id')
    status = data.get('status')
    started_at = data.get('started_at')
    expires_at = data.get('expires_at')
    cancelled_at = data.get('cancelled_at')
    failed_at = data.get('failed_at')
    completed_at = data.get('completed_at')
    required_action = data.get('required_action')
    last_error = data.get('last_error')
    model = data.get('model')
    instructions = data.get('instructions')
    tools = data.get('tools')
    tool_resources = data.get('tool_resources')
    metadata = data.get('metadata')
    temperature = data.get('temperature')
    top_p = data.get('top_p')
    max_completion_tokens = data.get('max_completion_tokens')
    max_prompt_tokens = data.get('max_prompt_tokens')
    truncation_strategy = data.get('truncation_strategy')
    incomplete_details = data.get('incomplete_details')
    usage = data.get('usage')
    response_format = data.get('response_format')
    tool_choice = data.get('tool_choice')
    parallel_tool_calls = data.get('parallel_tool_calls')

    # Exibindo os valores das variáveis (opcional)
    print("Run ID:", run_id)
    print("Object Type:", object_type)
    print("Created At:", created_at)
    print("Assistant ID:", assistant_id)
    print("Thread ID:", thread_id)
    print("Status:", status)
    print("Started At:", started_at)
    print("Expires At:", expires_at)
    print("Cancelled At:", cancelled_at)
    print("Failed At:", failed_at)
    print("Completed At:", completed_at)
    print("Required Action:", required_action)
    print("Last Error:", last_error)
    print("Model:", model)
    print("Instructions:", instructions)
    print("Tools:", tools)
    print("Tool Resources:", tool_resources)
    print("Metadata:", metadata)
    print("Temperature:", temperature)
    print("Top P:", top_p)
    print("Max Completion Tokens:", max_completion_tokens)
    print("Max Prompt Tokens:", max_prompt_tokens)
    print("Truncation Strategy:", truncation_strategy)
    print("Incomplete Details:", incomplete_details)
    print("Usage:", usage)
    print("Response Format:", response_format)
    print("Tool Choice:", tool_choice)
    print("Parallel Tool Calls:", parallel_tool_calls)

else:
    print("Erro ao enviar dados:", response.status_code, response.text)


In [ ]:
# Pega Status
url = f'https://api.openai.com/v1/threads/{thread_id}/runs/{run_id}'

# 5. Enviar a requisição POST com o corpo em JSON
response = requests.post(url, headers=headers)

# 6. Verificar a resposta
# Supondo que 'response' é a resposta da requisição POST já verificada
if response.status_code == 200:
    data = response.json()
    
    # Extraindo os parâmetros em variáveis
    run_id = data.get('id')
    object_type = data.get('object')
    created_at = data.get('created_at')
    assistant_id = data.get('assistant_id')
    thread_id = data.get('thread_id')
    status = data.get('status')
    started_at = data.get('started_at')
    expires_at = data.get('expires_at')
    cancelled_at = data.get('cancelled_at')
    failed_at = data.get('failed_at')
    completed_at = data.get('completed_at')
    required_action = data.get('required_action')
    last_error = data.get('last_error')
    model = data.get('model')
    instructions = data.get('instructions')
    tools = data.get('tools')
    tool_resources = data.get('tool_resources')
    metadata = data.get('metadata')
    temperature = data.get('temperature')
    top_p = data.get('top_p')
    max_completion_tokens = data.get('max_completion_tokens')
    max_prompt_tokens = data.get('max_prompt_tokens')
    truncation_strategy = data.get('truncation_strategy')
    incomplete_details = data.get('incomplete_details')
    usage = data.get('usage')
    response_format = data.get('response_format')
    tool_choice = data.get('tool_choice')
    parallel_tool_calls = data.get('parallel_tool_calls')

    # Exibindo os valores das variáveis (opcional)
    print("Run ID:", run_id)
    print("Object Type:", object_type)
    print("Created At:", created_at)
    print("Assistant ID:", assistant_id)
    print("Thread ID:", thread_id)
    print("Status:", status)
    print("Started At:", started_at)
    print("Expires At:", expires_at)
    print("Cancelled At:", cancelled_at)
    print("Failed At:", failed_at)
    print("Completed At:", completed_at)
    print("Required Action:", required_action)
    print("Last Error:", last_error)
    print("Model:", model)
    print("Instructions:", instructions)
    print("Tools:", tools)
    print("Tool Resources:", tool_resources)
    print("Metadata:", metadata)
    print("Temperature:", temperature)
    print("Top P:", top_p)
    print("Max Completion Tokens:", max_completion_tokens)
    print("Max Prompt Tokens:", max_prompt_tokens)
    print("Truncation Strategy:", truncation_strategy)
    print("Incomplete Details:", incomplete_details)
    print("Usage:", usage)
    print("Response Format:", response_format)
    print("Tool Choice:", tool_choice)
    print("Parallel Tool Calls:", parallel_tool_calls)
    print('#'*10)
    print(response.text)
else:
    print("Erro ao enviar dados:", response.status_code, response.text)

In [ ]:
thread_id

In [ ]:
import requests

# URL da API
url = f'https://api.openai.com/v1/threads/{thread_id}/messages?order=desc'

# Enviar a requisição GET
response = requests.get(url, headers=headers)

# Verificar a resposta
if response.status_code == 200:
    data = response.json().get('data', [])  # Pega a lista de mensagens, se existir
    
    # Verifica se há mensagens
    if data:
        last_assistant_message = None  # Inicializa a variável para a última mensagem do assistente

        for message in data:
            role = message.get('role')
            if role == 'assistant':
                last_assistant_message = message.get('content')  # Atualiza a última mensagem do assistente
        
        # Imprime a última mensagem do assistente, se existir
        if last_assistant_message:
            # O conteúdo está em formato de lista, precisamos acessar o texto
            content = last_assistant_message[0]['text']['value'] if isinstance(last_assistant_message, list) and last_assistant_message else "Nenhum conteúdo encontrado."
            print(content)
        else:
            print("Nenhuma mensagem do assistente encontrada.")
    else:
        print("Nenhuma mensagem encontrada.")
else:
    print("Erro ao buscar mensagens:", response.status_code, response.text)


In [ ]:
if content:
    print('ok')
    